In [1]:
# imports

macaulay2('loadPackage("PrimaryDecomposition", Reload=>true)')
macaulay2('loadPackage("Depth", Reload=>true)')

import re

In [2]:
def get_indeterminates(ideal):
    '''
    ideal a string that is either an m2 ideal, such as 'ideal(x*y)', or an m2 list of generators, '{x*y}'
    returns a (python) list of strings; each string a single indeterminate appearing in `ideal`
    '''    
    # trim the ideal to get the generators
    if ideal.startswith('ideal('):
        ideal = ideal[6:]
    elif ideal.startswith('{'):
        ideal = ideal[1:]
    if ideal.endswith(')') or ideal.endswith('}'):
        ideal = ideal[:-1]
        
    # remove all whitespace
    ideal = ideal.replace(' ','')
    
    # remove exponentiation
    ideal = re.sub("\^[0-9]+", '', ideal)
    
    # split over addition, subtraction, multiplication, distribution, and commas
    gens_list = re.split(',|\*|\+|-', ideal)
    
    # remove ``''``, ``None``, and any "stuck" parentheses, i.e., `(x_1` -> `x_1`
    gens = [gen.replace('(','').replace(')','') for gen in gens_list if gen != '' and gen is not None]
    
    # filter down to unique occurrences and sort
    unique_gens = sorted(list(set(gens)))
    
    # remove numbers
    indets = [i for i in unique_gens if not i.isdigit()]
    
    return indets


def ring_from_indets(indets, weighted_indet='first'):
    '''
    indets: list of indeterminates
    weighted_indet: the indeterminate to be weighted 10, others will have weight 0
    if weighed_indet == None, sets it to the first indet
    
    returns string of m2 ring with weighted_indet weighted 10, others weighted 0
    '''
    
    indet_str = ','.join(indets)
    
    if weighted_indet is None:
        return f'QQ[{indet_str}]'  # default M2 ordering
    
    if weighted_indet == 'first':
        # if weighted_indet is not specified, set it as the first indeterminate in the ring
        weighted_indet = indets[0]

    weights = make_weights(indets, weighted_indet)
    ring = 'QQ[' + indet_str +', MonomialOrder=>{Weights=>' + weights + '}, Global=>false]'
    return ring


def make_weights(indets, weighted_indet, min_weight=0, max_weight=10):
    '''
    makes an M2 list of weights; one weight per indeterminate
    
    used below for 1) creating the ring with custom monomial order, 
    and 2) for getting the initial y-form    
    
    examples
    --------
    >> make_weights(['x', 'y', 'z'], 'y')
    {0, 10, 0}
    
    >> make_weights(['w', 'r', 's'], min_weight=10, max_weight=3)
    {3, 10, 10}
    '''
    
    weights = ''  # output string
    min_weight = str(min_weight)
    max_weight = str(max_weight)
    
    for elt in indets:
        if elt == weighted_indet:
            weights += max_weight + ','
        else:
            weights += min_weight + ','
            
    weights = weights[:-1]  # remove trailing comma
    return '{' + weights + '}'


def getC(ring, init_form, y):
    '''
    for an ideal $I$ in the ring ``ring`` and indeterminate ``y`` with initial y-form
    ``init_form``, returns $C_{y,I}$
    
    uses a remark from GVD & Liaison (Klein, Rajchgot) after Definition 2.3
    where the $C_{y,I}$ is defined by the saturation $C_{y,I} = in_y(I) : y^\infty$
    '''
    macaulay2.set('R', ring)
    macaulay2.set('initYForm', init_form)
    C = str(macaulay2(f'toString saturate(initYForm, ideal({y}))'))
    return C


def getN(gb, y):
    '''
    for an ideal $I$ with Groebner basis ``gb`` and indeterminate ``y``,
    returns the ideal $N_{y,I}$
    
    the generators of $N_{y,I}$ are the elements of ``gb`` that
    do not include ``y``
    '''    
    if not squarefree_ideal_in_y(gb, y):
        return False
    
    # the elements of the Groebner basis
    # note: this uses the fact that the GB is stored as an ideal
    gb_elts = [elt.strip() for elt in gb[6:-1].split(',')]
    N_gens = ''
    
    for elt in gb_elts:
        indets = get_indeterminates(f'ideal({elt})')
        
        if y not in indets:  # then ``elt`` is a generator of $N_{y,I}$
            if N_gens != '':
                N_gens += ','
            N_gens += elt
        
    N = f'ideal({N_gens})'  # store as an ideal
    return N


def valid_gvdecomp(C, N, y, init_form, ring):
    '''
    checks if $C \cap (N + \langle y \rangle) = in_y(I)$
    requires C, N, init_form to be ideals
    
    returns ``True`` if the equality holds, ``False`` otherwise
    '''
    macaulay2.set('R', ring)
    macaulay2.set('inYForm', init_form)
    macaulay2.set('C', C)
    macaulay2.set('Nplus', f'{N} + ideal({y})')

    # check equality of the intersection
    # use `sub` to ensure the unit/zero ideals are viewed in ``R``
    output = bool(macaulay2('intersect(sub(C, R), sub(Nplus, R)) == inYForm'))
    return output


def is_squarefree(ideal):
    '''
    returns ``True`` if the ideal ``ideal`` is squarefree, ``False`` otherwise
    '''
    return "^" not in ideal


def squarefree_in_y(poly, y):
    '''
    returns True if the polynomial ``poly`` is squarefree in ``y``, some indeterminate
    '''
    # test if entire polynomial is squarefree
    if is_squarefree(poly):
        return True
    
    # if there is a square, check if ``y`` is squared
    squares = re.finditer("\^", poly)
    squares_indices = [m.span() for m in squares]
    squared_indets = list(set([poly[ind[0]-len(y):ind[1]-1] for ind in squares_indices]))
    
    return y not in squared_indets


def squarefree_ideal_in_y(ideal, y):
    '''
    returns ``True`` if the ideal ``ideal`` is squarefree in the indeterminate ``y``,
    and returns ``False`` otherwise
    '''
    if ideal == 'ideal()':
        return True
    
    generators = [gen.strip() for gen in ideal[6:-1].split(',')]
    is_gen_squarefree = [squarefree_in_y(gen, y) for gen in generators]
    # return ``True`` if all generators are squarefree in ``y``
    return False not in is_gen_squarefree


def is_unit_ideal(ideal):
    '''
    returns ``True`` if the ideal ``ideal`` is the unit ideal, returns ``False`` otherwise
    '''
    ring = ring_from_indets(get_indeterminates(ideal), weighted_indet=None)
    macaulay2.set('R', ring)
    macaulay2.set('i', ideal)
    is_unit = bool(macaulay2('i == 1'))
    return is_unit


def gens_are_indets(ideal):
    '''
    returns ``True`` if an ideal is generated by indeterminates;
    returns False otherwise
    '''
    if ideal == 'ideal()':  # zero ideal
        return True
    
    if ideal == 'ideal 1':  # unit ideal
        return False
    
    if not is_squarefree(ideal):
        return False
    
    if '(' not in ideal:
        # generated by a single indeterminate
        return True
    
    ideal_interior = ideal[6:-1]
    gens = [gen.strip() for gen in ideal_interior.split(',')]
    
    for gen in gens:
        # check if there is addition, multiplication, subtraction, 
        # or exponentiation in the generator
        if len(re.findall('\*|\+|-|\^', gen)) > 0:
            return False
    return True


def is_unmixed(ideal, ring):
    '''
    ideal: str, an m2 ideal, assumed not to be either the unit or zero ideal
    
    returns ``True`` if the ideal ``ideal`` is unmixed
    '''
    macaulay2.set('R', ring)
    macaulay2.set('I', f'sub({ideal}, R)')
    macaulay2.set('D', 'primaryDecomposition I')
    macaulay2.set('d', 'apply(D, i->dim(R/i))')  # get dimension of each component
    # check that each dimension is equal to the dimension of the first component
    dimensions = str(macaulay2('toString(apply(d, i->(i==d_0)))'))
    return "false" not in dimensions


def is_homogeneous(ideal, ring):
    '''
    Returns ``True`` if the ideal ``ideal`` is homogeneous; ``False`` otherwise
    
    uses Macaulay2 to check the homogeneity
    '''
    macaulay2.set('R', ring)
    macaulay2.set('I', ideal)
    is_hom = bool(macaulay2('isHomogeneous I'))
    return is_hom


def print_if(print_bool, string):
    '''
    prints ``string`` if ``print_bool`` is ``True``; does nothing otherwise
    
    used to implement the ``show_output`` parameter in the ``isGVD`` method
    '''
    if print_bool:
        print(string)

        
def order_indeterminates(ordered_indets, all_indets):
    '''
    returns a list of all indeterminates, with the indeterminates
    in ``ordered_indets`` followed by the remaining indeterminates in 
    ``all_indets`` $\setminus$ ``ordered_indets``
    '''
    
    if ordered_indets is None:
        return all_indets
    
    # remove indeterminates from ``ordered_indets`` if they're not in ``all_indets`` 
    ordered_indets_trimmed = [x for x in ordered_indets if x in all_indets]
    
    # append remaining indeterminates
    indet_list = ordered_indets_trimmed + \
                [x for x in all_indets if x not in ordered_indets_trimmed]
    return indet_list    


def do_one_gvd_step(ideal, indet, show_output=True):
    '''
    returns the $C$, $N$ ideals given the ideal ``ideal`` 
    and some indeterminate in the ideal ``indet``
    
    we print errors if $C$, $N$ don't form a gvd or 
    if either $C$ or $N$ is unmixed
    '''
    # get indeterminates
    indets_in_ideal = get_indeterminates(ideal)
    
    # check ``indet`` appears in the ideal
    if indet not in indets_in_ideal:
        print_if(show_output, f'[ERROR] {indet} does not appear in the ideal')
        return False, False
    
    # set up the ring 
    m2_ring = ring_from_indets(indets_in_ideal, indet)
    macaulay2.set('R', m2_ring)

    # get a Groebner basis
    macaulay2.set('i', ideal)
    macaulay2.set('G', 'ideal gens gb i')  # stored as an ideal
    gb = str(macaulay2('toString G'))

    # get the initial y-form
    init_y_form = str(macaulay2('toString ideal leadTerm(1,i)'))    
    
    # get C and N
    C, N = getC(m2_ring, init_y_form, indet), getN(gb, indet)
    
    # if either is ``False``, then GB not squarefree in ``y``
    if C == False or N == False:
        print_if(show_output, f'[WARNING] Groebner basis not squarefree in {indet}')
        return False, False
    
    print_if(show_output, f'C = {C}\nN = {N}')
    
    # check that $C$, $N$ form a geometric vertex decomposition
    if not valid_gvdecomp(C, N, indet, init_y_form, m2_ring):
        print_if(show_output, '[WARNING] C and N do not form a valid geometric vertex decomposition')
        return False, False

    print_if(show_output, 'C and N form a valid geometric vertex decomposition')
    
    # check unmixedness of $C$, $N$
    C_unmixed = is_unmixed(C, m2_ring)
    N_unmixed = is_unmixed(N, m2_ring)
    
    # print warnings
    if not C_unmixed:
        print_if(show_output, '[WARNING] C is not unmixed')
    elif not N_unmixed:
        print_if(show_output, '[WARNING] N is not unmixed')
    
    # and now return ``False`` if either are not unmixed
    if not C_unmixed or not N_unmixed:
        return False, False
    
    return C, N


def isGVD(ideal, indet_order=None, show_output=True, homogeneous=False, check_CM=True, check_unmixed=True):
    '''
    ideal: str, an M2 ideal
    returns ``True`` if ``ideal`` is geometrically vertex decomposable
    '''
    print_if(show_output, f'------\nI = {ideal}')
    
    if indet_order == [] or indet_order == ():
        print_if(show_output, 'No ordering given; reverting to default behaviour')
        indet_order = None
    
    if ideal == 'ideal()':
        print_if(show_output, 'zero ideal')
        return True  # vacuously generated by indeterminates
    if is_unit_ideal(ideal):
        print_if(show_output, 'unit ideal')
        return True  # is the whole ring
    
    # check the case where `ideal` is generated by a single indeterminate,
    if '(' not in ideal:  # only one indeterminate generates the ideal
        if "^" not in ideal:  # squarefree case
            print_if(show_output, 'generated by a single indeterminate')
            return True
        else:
            # the ideal is of the form `ideal x^2`
            print_if(show_output, 'not GVD')
            return False
    
    ideal = ideal.replace(' ','')  # remove all whitespace
    
    indets_in_ideal = get_indeterminates(ideal)
    default_ring = ring_from_indets(indets_in_ideal, weighted_indet=None)
    
    # check unmixedness
    if not is_unmixed(ideal, default_ring):  
        # assumes that `ideal` is neither the zero ideal nor the unit ideal       
        # checks using default m2 monomial ordering
        print_if(show_output, 'ideal is not unmixed => not GVD')
        return False
    
    # check if the ideal is generated by indeterminates
    if gens_are_indets(ideal):
        print_if(show_output, 'generated by indeterminates')
        return True
    
    # not CM & homogeneous & proper => not GVD
    if not homogeneous:
        macaulay2.set('R', default_ring)
        macaulay2.set('I', ideal)
        homogeneous = bool(macaulay2('isHomogeneous I'))
    # by default, only check CM on first ideal
    # set parameter ``check_CM='always'`` to always check
    if check_CM is True or check_CM == 'always': 
        if homogeneous and not bool(macaulay2('isCM(R/I)')):
            print_if(show_output, 'ideal is homogeneous, proper, and not CM => not GVD')
            return False

    # iterate over all choices of indeterminates
    indeterminates = order_indeterminates(indet_order, indets_in_ideal)
    for indet in indeterminates:
        
        print_if(show_output, f'decomposing with respect to {indet}')
        
        C, N = do_one_gvd_step(ideal, indet, show_output=show_output)
        
        # C, N are ``False`` if not unmixed, not valid GVD, 
        # or GB not squarefree in ``indet``
        if not C or not N:
            if indet_order is not None:
                print_if(show_output, 'not GVD with the given ordering')
                return False
            else:
                continue
        
        check_CM = str(check_CM).lower() == 'always'
        
        # unmixedness is already checked in ``do_one_gvd_step``
        # homogeneity of $I$ implies homogeneity of $C_{y,I}$, $N_{y,I}$
        if indet_order is None:
            is_C_GVD = isGVD(C, show_output=show_output, homogeneous=homogeneous, check_CM=check_CM, check_unmixed=False)
            is_N_GVD = isGVD(N, show_output=show_output, homogeneous=homogeneous, check_CM=check_CM, check_unmixed=False)
        else:
            is_C_GVD = isGVD(C, indet_order=indet_order[1:], show_output=show_output, homogeneous=homogeneous, check_CM=check_CM, check_unmixed=False)
            is_N_GVD = isGVD(N, indet_order=indet_order[1:], show_output=show_output, homogeneous=homogeneous, check_CM=check_CM, check_unmixed=False)

        if is_C_GVD and is_N_GVD:
            return True
        elif indet_order is not None:
            return False

    return False  # after checking all indeterminates

In [3]:
# test 1 (returns True)

isGVD('ideal(y*(z*s-x^2), y*w*r, w*r*(z^2+z*x+w*r+s^2))', show_output=False)

True

In [4]:
# test 2 (returns False)

isGVD('ideal(x^2*y, y*z, x*z)', indet_order=['x', 'y', 'z'])

------
I = ideal(x^2*y, y*z, x*z)
decomposing with respect to x
[WARNING] Groebner basis not squarefree in x
not GVD with the given ordering


False